In [1]:
//Table creation
.bath.craigFlatTable:`$":/sharedfolder/cbath/Udacity_Bath/sample_DMCIPListening"
.bath.craigDisplayTable:`$":/sharedfolder/cbath/Udacity_Bath/sample_displayClicks"

In [ ]:
// Load IP2Location sym file and apply unique attribute
load`$"<REDACTED>";
ipSym:`u#ipSym;
//Load IP2Location table
load`$"<REDACTED>";
// Memory map IP2Location table
`IP2Location set -23!latest;

In [ ]:
// Load AMDB sym file and apply unique attribute
load`$"<REDACTED>";
amdbSym:`u#amdbSym;
//Load AMDB table
load`$"<REDACTED>";
// Memory map accountRooftop table
`accountRooftop set -23!accountRooftop;

In [3]:
// Load keyword activity data directory
system"l <REDACTED>"

In [4]:
//Setup parameters
days:31;
startDate: 2019.03.01;
dates: startDate + til days;
(first dates;last dates)

"Getting data for 2019.03.01"


2019.03.01 2019.03.31


In [6]:
//Create base data from DMCIPListening
.bath.createDateSample:{[x]
    start:.z.p;
    show"Getting data for ",string[x];
    base:select hits:count i, sum pageViews,uniqueViews: sum unique
         by  dates:date, domains: domain, company,userDomain,city,region:state, countryCode:country,iabCat
         from <REDACTED>
         where date =x;    
    //Remove enumeration
    base:@[0!base;exec c from meta[base] where t="s";value];
    //Convert to parent IAB category for analysis
    parentCat:{`$"IAB_",/: ("-" vs/: string x)[;0]};
    base:update iabCat:.Q.fu[parentCat;`unknown^iabCat] from base;
    show"Gotten  data for ",string[x],". Rows returned = ", string[count base],". Process time = ", string .z.p - start;
    :base;
 };

//Create initial flat table
.bath.craigFlatTable set .bath.createDateSample[first dates]


"Gotten  data for 2019.03.01. Rows returned = 2822119. Process time = 0D00:00:08.203080000"


`:/app/prelytix/kxinstall/delta-data/cbath/Udacity/sample_DMCIPListening


"Processing..."
"Getting data for 2019.03.02"


In [ ]:
//Upsert following dates individually to prevent memory issyes
show"Processing..."
.bath.createFull:{[date]
    .bath.craigFlatTable upsert .bath.createDateSample[date];
    .Q.gc[];
 };
.bath.createFull each dates except first dates
show"Processed"

"Gotten  data for 2019.03.02. Rows returned = 1028807. Process time = 0D00:00:02.514452000"
"Getting data for 2019.03.03"
"Gotten  data for 2019.03.03. Rows returned = 912680. Process time = 0D00:00:02.297354000"
"Getting data for 2019.03.04"
"Gotten  data for 2019.03.04. Rows returned = 2984996. Process time = 0D00:00:08.580888000"
"Getting data for 2019.03.05"
"Gotten  data for 2019.03.05. Rows returned = 3194964. Process time = 0D00:00:09.349600000"
"Getting data for 2019.03.06"
"Gotten  data for 2019.03.06. Rows returned = 3198318. Process time = 0D00:00:09.338185000"
"Getting data for 2019.03.07"


In [ ]:
//Load Click data
system"l <REDACTED>";

In [ ]:
//Small enough to manage in-memory
base:update `s#ipStart from `ipStart xasc select clicks:count i, clickDates:count distinct date by ipStart:ip from <REDACTED> where date in dates
//Grab location details from IP address
base:delete ipStart from aj[`ipStart;base;select ipStart,org,domain,city,region,countryCode  from <REDACTED>]

.bath.craigDisplayTable set  base:@[0!base;exec c from meta[base] where t="s";value]